In [ ]:
# Import all python modules
from jupyter_plotly_dash import JupyterDash

import sys, os
sys.path.append(os.path.expanduser('~/Documents'))
from crud import AnimalShelter

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bson.json_util import dumps
from bson.json_util import loads
import json

# MongoDB authentication and query
shelter = AnimalShelter('AAC', 'aacuser', 'Suki!007')
df = pd.DataFrame.from_records(list(shelter.find('animals',{})))

app = dash.Dash('ModuleSixMilestone')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(children=[
                    html.A(href='http://www.snhu.edu',
                           children=[
                               html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                style={'height': '120px'})]),
                    html.B(html.H1('SNHU CS-340 Dashboard')),
                    ]),
    html.Center(html.H5('Sam Brasher Interactive Data Table')),
    html.Hr(),
    # Our buttons to let the user sort by animal type
    html.Div(className='row',
            style={'display': 'flex'},
                children=[
                    dcc.RadioItems(
                        id='radio-options',
                        options=[
                            {'label': 'Water', 'value': 'water_option'},
                            {'label': 'Mountain or Wilderness', 'value': 'mount_option'},
                            {'label': 'Disaster or Individual Tracking', 'value': 'dis_option'},
                            {'label': 'Reset', 'value': 'reset_option'},
                        ],
                        value='reset_option',
                        style={'font-size':'12px'}
                    )
                ]),
    # Start of DataTable
    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns
        ],
        editable=False,
        filter_action="native",
        sort_action="native",
        column_selectable=False,
        # Single selectable row to allow user to display different animals in the geolocation chart.
        row_selectable='single',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10
    ),
    html.Br(),
    # Start of geolocation and pie chart
    html.Hr(),
    html.Div(
        dcc.Graph(id="pieChart"),
        style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Div(
        id='map-id',
        className='col s12 m6',
        style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Br(),
    html.Hr(),
    
])

@app.callback(Output('datatable-id',"data"),
             Input('radio-options', 'value'))
# The callback function from button presses that sort animal type.
# It takes the id of the button pressed to determine what animal to sort by.
def on_click(value):
    df = pd.DataFrame.from_records(list(shelter.find('animals',{})))
    if value == 'water_option':
        df = pd.DataFrame.from_records(list(shelter.find('animals',{"breed": re.compile('Labrador|Chesa|Newfoundland', re.IGNORECASE),
                                                                   "sex_upon_outcome": "Intact Female",
                                                                   "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}
                                                                   })))
    elif value == 'mount_option':
        df = pd.DataFrame.from_records(list(shelter.find('animals',{"breed": re.compile('German Shepherd|Alaskan Mala|Old English Sheepdog|Siberian|Rottweiler', re.IGNORECASE),
                                                                   "sex_upon_outcome": "Intact Male",
                                                                   "age_upon_outcome_in_weeks": {"$lte": 156, "$gte": 26}
                                                                   })))
    elif value == 'dis_option':
        df = pd.DataFrame.from_records(list(shelter.find('animals',{"breed": re.compile('Doberman Pinsch|German Shepherd|Golden|Bloodhound|Rottweiler', re.IGNORECASE),
                                                                   "sex_upon_outcome": "Intact Male",
                                                                   "age_upon_outcome_in_weeks": {"$lte": 300, "$gte": 20}
                                                                   })))
    elif value == 'reset_option':
        df = pd.DataFrame.from_records(list(shelter.find('animals',{})))
        
    return df.to_dict('records')
    
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "derived_viewport_selected_rows"))
# The callback function for the map takes the first entry from whatever page is currently loaded
# and displays its lat/lon data in a marker with a popup for breed and name.
def update_map(viewData, selectedRow):
    dff = pd.DataFrame.from_dict(viewData)
    if selectedRow is None:
        selectedRow = []
        row = 0
    else:
        row = selectedRow[0]
        
    newLat = dff.loc[row,'location_lat']
    newLon = dff.loc[row,'location_long']
    # Austin, TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '800px', 'height': '500px'}, center=[newLat,newLon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[newLat,newLon], children=[
                dl.Tooltip(dff.loc[row,'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[row,'name'])
                ])
            ])
        ])
    ]

@app.callback(
    Output("pieChart", "figure"),
    Input('datatable-id', 'derived_virtual_data'),
    Input('radio-options', 'value'))
# This callback function updates the piechart based on the data in the datatable.
def generate_chart(viewData, value):
    # If user clicks "Reset", we display breakdown by breed to present a nicer pie chart.
    option = 'animal_type' if value=='reset_option' else 'breed'
    graph_title = 'Species Breakdown' if value=='reset_option' else 'Breed Breakdown'
    dff = df if pd.DataFrame.from_dict(viewData) is None else pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names = option, hole=.3, title = graph_title)
    return fig

if __name__=='__main__':
    app.run_server()